#  Delta Time Travel Demo
* Demonstrate OSS Delta time travel features
* This notebook works both as a Jupyter or Databricks notebook
* Jupyter
  * For Jupyter we assume you have Spark installed on your laptop
  * Works with OSS Delta
  * Note: Delta SQL does not work in OSS
* Databricks
  * For Databricks path names are `dbfs:`
  * Works with full Databricks Delta

### Prelude

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TimeTravel").getOrCreate()

In [ ]:
import os, sys, time
sys.version

In [ ]:
os.environ.get('PYSPARK_SUBMIT_ARGS',None)

In [ ]:
in_databricks = 'DATABRICKS_RUNTIME_VERSION' in os.environ
in_databricks

### Setup

In [ ]:
table = "cats"
if in_databricks:
  user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().get("user").get()
  user = user.split("@")[0].replace(".","_")
  database = f"{user}_delta_fun"
  dataPath = f"dbfs:/tmp/{user}/delta_fun/table_cats"
else:
  database = "delta_fun"
  dataPath = "delta_fun"
database,dataPath

In [ ]:
def now(): 
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))
now()

In [ ]:
def insert(path, data, mode="append"):
    df = spark.createDataFrame(data, ["id","name","region"])
    df.coalesce(1).write.mode(mode).format("delta").save(path)

### Insert three batches of data

In [ ]:
insert(dataPath, [
    (1, "lion","africa"),
    (2, "cheetah","africa"),
    (3, "leopard","africa")], "overwrite")

In [ ]:
insert(dataPath, [
    (4, "jaguar","south america"),
    (5, "puma","south america"),
    (6, "ocelot","south america")])

In [ ]:
insert(dataPath, [
    (7, "lynx","north america"),
    (8, "bobcat","north america"),
    (9, "catamount","north america")])

### Check data files

In [ ]:
if in_databricks:
  for f in dbutils.fs.ls(dataPath):
    print(f.size,f.name)
else:
  for f in os.listdir(dataPath):
    print(f)

### Display all data

In [ ]:
df = spark.read.format("delta").load(dataPath)
df.sort("id").show()

### Display versioned data

In [ ]:
for v in range(0,3):
    print("Version",v)
    df = spark.read.format("delta").option("versionAsOf", str(v)).load(dataPath)
    df.sort("id").show(100,False)

### Use Delta with SQL

In [ ]:
spark.sql(f"create database if not exists {database}")
spark.sql(f"use {database}")
spark.sql(f"drop table if exists {table}")
df = spark.sql("show databases").filter(f"databaseName = '{database}'")
df.show()

In [ ]:
spark.sql(f"create table {table} using delta location '{dataPath}'")

In [ ]:
spark.sql(f"describe formatted {table}").show(1000,False)

In [ ]:
spark.sql(f"select * from {table} order by id").show()

#### Databricks-specific queries
These do not work on OSS Delta.

In [ ]:
if in_databricks: spark.sql(f"describe history {table}").show()

In [ ]:
if in_databricks: spark.sql(f"describe detail {table}").show()

In [ ]:
if in_databricks: 
    for v in range(0,3):
        print("Version",v)
        spark.sql(f"select * from {table} version as of {v} order by id").show()

In [ ]:
now()